In [1]:
import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from medusa.data import get_example_h5
from medusa.preproc import align
from copy import deepcopy
plt.rcParams['figure.facecolor'] = 'white'
import numpy as np

In [2]:
data = get_example_h5(load=True, model='mediapipe')
data_aligned = align(deepcopy(data))

from medusa.render import Renderer
renderer = Renderer(data.img_size, camera_type='intrinsic', smooth=False)

2022-08-15 15:26 [INFO   ]  Align frames: 100%|██████████| 232/232 [00:00<00:00, 6982.53it/s]


In [ ]:
glob_mot = data.decompose_mats()
glob_mot = glob_mot - glob_mot.iloc[0, :]
trans = glob_mot.loc[:, ['Trans. X', 'Trans. Y', 'Trans. Z']]
rot = glob_mot.loc[:, ['Rot. X (deg)', 'Rot. Y (deg)', 'Rot. Z (deg)']]
rot.columns = ['Rot. X', 'Rot. Y', 'Rot. Z']

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(trans, lw=3)
ax.set_ylabel('Translation (mm.)', fontsize=20)
ax.set_ylim(-4, 4)
ax.set_yticks([-4, 0, 4])
ax.tick_params(axis='y', labelsize=15)

ax.legend(trans.columns, frameon=False, fontsize=15, loc=2)
ax.axhline(0., ls='--', c='k')

ax2 = ax.twinx()
ax2.set_ylabel('Rotation (deg.)', fontsize=20)
ax2.set_ylim(-30, 30)
ax2.set_yticks([-30, 0, 30])
ax2.tick_params(axis='y', labelsize=15)

ax2.plot(rot, lw=3, ls='-.')
ax2.legend(rot.columns, frameon=False, fontsize=15, loc=3)

ax2.set_xlim(-1, glob_mot.shape[0])
ax.spines['top'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.set_xlabel('Time (frames)', fontsize=20)
ax2.set_xlabel('Time (frames)', fontsize=20)

In [ ]:
v = data_aligned.v
v = v - v[0, :, :]
mask = np.abs(v * 10).max(axis=0) > 8

fig, ax = plt.subplots(figsize=(15, 5))
plt.plot(v[:, np.random.randint(468, size=5), 0], c='tab:orange')
plt.plot(v[:, np.random.randint(468, size=5), 1], c='tab:blue')
plt.plot(v[:, np.random.randint(468, size=5), 2], c='tab:green')
#plt.plot(v[:, mask[:, 0], 0], c='r')
#plt.plot(v[:, mask[:, 1], 1], c='g')
#plt.plot(v[:, mask[:, 2], 2], c='b')


In [3]:
from trimesh.geometry import mean_vertex_normals
from trimesh.triangles import normals, cross

triangles = data_aligned.v[0, :, :][data_aligned.f]
crosses = cross(triangles)
face_normals, valid = normals(triangles, crosses)
vnorms = mean_vertex_normals(468, data_aligned.f, face_normals)

In [25]:
pca = PCA(n_components=5)
v = data_aligned.v.reshape(len(data), -1) * 10
v = v - v[0, :]
v_pca = pca.fit_transform(v)
overlay = (pca.components_[3, :].reshape((468, 3)) * vnorms).sum(axis=1)
face_colors = overlay[data_aligned.f].mean(axis=1)
face_colors = (face_colors - face_colors.min()) / (face_colors.max() - face_colors.min())
face_colors = plt.get_cmap('bwr')(face_colors)

In [27]:
pca = PCA(n_components=5)
v = data_aligned.v.reshape(len(data), -1)
v = v - v[0, :]
v_pca = pca.fit_transform(v)
overlay = (pca.components_[3, :].reshape((468, 3)) * vnorms).sum(axis=1)
overlay = overlay.round(2) * 10 + 0.5
overlay_ = plt.get_cmap('bwr')(overlay)
overlay_[np.logical_and(overlay < 0.6, overlay > 0.4)] = [0.5, 0.5, 0.5, 0]
img = renderer(data.v[0, :, :], data.f, overlay=face_colors)
plt.imshow(img)
plt.axis('off')

ValueError: Cannot infer whether overlay refers to vertices or faces (polygons)!

In [28]:
face_colors.shape

(898, 4)